In [2]:
## ======= ##
## IMPORTS ##
## ======= ##

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import zarr
import rioxarray
import matplotlib.pyplot as plt
import os
import dask
import dask.array
import math

import datetime

from collections import Counter

import pystac_client
from pystac.extensions.projection import ProjectionExtension as proj

import planetary_computer
import rasterio
import rasterio.features
from rasterio.features import rasterize

import stackstac
import pyproj

import dask.diagnostics

from shapely.geometry import box
from shapely.ops import transform

from scipy.ndimage import binary_propagation
from scipy.ndimage import label

import sat_tile_stack
from sat_tile_stack import sattile_stack, sat_mask_array, write_netcdf_from_da
from sat_tile_stack import *



In [18]:
## =============================================================================== ##
## LOOP OVER LAKES IN THE .csv FILE AND SAVE EACH TIMESTACK AS A SEPARATE .nc FILE ##
## =============================================================================== ##

from tqdm.auto import tqdm
import time, datetime as dt

# READ IN THE LAKE INFORMATION .geojson FILE
gdf = gpd.read_file("/home/jupyter/repos/sat-tile-stack/gdfs/labels_2019_volumes.geojson")
gdf['centroid'] = gdf.geometry.centroid
gdf['centroid_x'] = gdf.centroid.x
gdf['centroid_y'] = gdf.centroid.y
gdf_mask = gdf

# DECIDE WHETHER TO NORMALIZE THE IMAGERY UPON COMPLIING OR NOT
normalize = False

# SPECIFY TIME RANGE
time_range = '2019-05-01/2019-09-30'

# SPECIFY REGION (PICK FROM: NW, CW, SW, SE, NE, NO)
region = 'CW'
gdf = gdf[gdf['region'] == region]

# SPECIFY YEAR (PICK FROM: 2018, 2019)
year = '2019'

## CONNECT TO MICROSOFT PLANETARY COMPUTER
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,)
print(f"connected to Microsoft Planetary Computer")

# SPECIFY IMAGERY BANDS
band_names = ["B04",  # red (665 nm)
              "B03",  # green (560 nm)
              "B02",  # blue (490 nm)
              "B08",  # NIR (842 nm)
              "B11",  # SWIR1 (1610 nm)
              "B12",  # SWIR2 (1.8 - 2.5 um)
             ]

total_start = time.perf_counter()

# LOOP OVER LAKE LOCATIONS
for idx_lake in tqdm(range(0,len(gdf)), desc="Processing lakes"):
    
    iter_start = time.perf_counter()

    print(f"\n working on lake {gdf.iloc[idx_lake].new_id}")

    # LAKE CENTROID
    centroid = (gdf.iloc[idx_lake].centroid_x, gdf.iloc[idx_lake].centroid_y)

    # CALL FUNCTION TO GENERATE TIMESTACK
    timestack = sattile_stack(catalog, centroid, band_names, pix_res=10, tile_size=512, time_range=time_range, normalize=False, cloudmask=True, mask=gdf_mask.iloc[[idx_lake]], pull_to_mem=True)

    # WRITE TIMESTACK AS .nc FILE TO DISK
    outfile = f"/home/jupyter/data/{region}{year}_tstacks/tstack_{gdf.iloc[idx_lake].new_id}.nc"
    os.makedirs(os.path.dirname(outfile), exist_ok=True) 
    write_netcdf_from_da(timestack, outfile)
    os.chmod(outfile, 0o777) # permissions
    
    tqdm.write(f"lake {gdf.iloc[idx_lake].new_id} → {dt.timedelta(seconds=time.perf_counter()-iter_start)}")
    
tqdm.write(f"all lakes → {dt.timedelta(seconds=time.perf_counter()-total_start)}")


connected to Microsoft Planetary Computer


Processing lakes:   0%|          | 0/1000 [00:00<?, ?it/s]


 working on lake CW2019_1524
epsg for bounds_latlon: 3413
pulling stack into memory, shape will be: (153, 8, 512, 512)
[########################################] | 100% Completed | 76.78 s
shape of stack in memory: (153, 8, 512, 512)
wrote /home/jupyter/data/CW2019_tstacks/tstack_CW2019_1524.nc
lake CW2019_1524 → 0:02:57.430145

 working on lake CW2019_1525
epsg for bounds_latlon: 3413
pulling stack into memory, shape will be: (153, 8, 512, 512)
[########################################] | 100% Completed | 43.44 s
shape of stack in memory: (153, 8, 512, 512)
wrote /home/jupyter/data/CW2019_tstacks/tstack_CW2019_1525.nc
lake CW2019_1525 → 0:01:40.074941

 working on lake CW2019_1526
epsg for bounds_latlon: 3413
pulling stack into memory, shape will be: (153, 8, 512, 512)
[########################################] | 100% Completed | 46.63 s
shape of stack in memory: (153, 8, 512, 512)
wrote /home/jupyter/data/CW2019_tstacks/tstack_CW2019_1526.nc
lake CW2019_1526 → 0:01:46.761808

 worki

RuntimeError: Error opening 'https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/W/EB/2019/05/15/S2B_MSIL2A_20190515T152819_N0212_R111_T22WEB_20201106T003737.SAFE/GRANULE/L2A_T22WEB_A011434_20190515T153250/IMG_DATA/R20m/T22WEB_20190515T152819_B12_20m.tif?st=2025-12-28T17%3A07%3A04Z&se=2025-12-29T17%3A52%3A04Z&sp=rl&sv=2025-07-05&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-12-29T05%3A45%3A07Z&ske=2026-01-05T05%3A45%3A07Z&sks=b&skv=2025-07-05&sig=G0oi1CXXzm1JA/d//8q%2BJhv%2BYqgoVLLyQPlGGmp%2B4O0%3D': RasterioIOError("'/vsicurl/https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/W/EB/2019/05/15/S2B_MSIL2A_20190515T152819_N0212_R111_T22WEB_20201106T003737.SAFE/GRANULE/L2A_T22WEB_A011434_20190515T153250/IMG_DATA/R20m/T22WEB_20190515T152819_B12_20m.tif?st=2025-12-28T17%3A07%3A04Z&se=2025-12-29T17%3A52%3A04Z&sp=rl&sv=2025-07-05&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&sk